# Import thư viện

In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 52.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from pyvi import ViTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from tqdm import tqdm
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Load dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Dữ liệu thông tin - mô tả của các khách sạn
hotel_data = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_info_hotel_new.csv')

In [25]:
# Dữ liệu lịch sử người dùng
df_his = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_history.csv')
# Dữ liệu rút gọn ID - tên khách sạn
df_hotel = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_hotel.csv')

# Lấy dữ liệu lịch sử cho mỗi người dùng
df_latest = pd.merge(df_his, df_hotel, on = 'IDhotel', how = 'inner')
df_latest = pd.merge(df_latest, hotel_data, on = 'NameHotel', how = 'inner')

In [26]:
# Lọc df_latest dựa vào ID của user
unique_user_ids = df_latest['IDuser'].unique()
filtered_df_latest = df_latest[df_latest['IDuser'].isin(unique_user_ids)]

# Nhóm dữu liệu theo ID
grouped = filtered_df_latest.groupby(filtered_df_latest['IDuser'])

fac_concat = grouped['Facilities'].apply(lambda x: ','.join(x.unique())).tolist()
ar_concat = grouped['Around'].apply(lambda x: ','.join(x.unique())).tolist()
vici_concat = grouped['Vicinity'].apply(lambda x: ','.join(x.unique())).tolist()
pri_concat = grouped['Price_Category'].apply(lambda x: ','.join(x.unique())).tolist()
name_hotels = grouped['NameHotel'].apply(lambda x: ','.join(x.unique())).tolist()

In [27]:
# DataFrame lịch sử người dùng
df_merge = pd.DataFrame({'IDuser': unique_user_ids,
                         'Name Hotel': name_hotels,
                         'Facilities': fac_concat,
                         'Around': ar_concat,
                         'Vicinity': vici_concat,
                         'Price': pri_concat})

df_merge

,IDuser,Name Hotel,Facilities,Around,Vicinity,Price
0,3040,Đức Trọng Hotel,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...","Tượng đài, Cafe/quán bar, Biển","Di tích, Khu cắm trại, Công viên, Biển, Sân ba...",Thấp
1,8755,TTR An Nam Apart Hotel,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...",Cafe/quán bar,"Vườn hoa, Quảng trường, Bảo tàng, Công Viên, D...",Thấp
2,7261,Hôtel Colline,"WiFi miễn phí, Phòng gia đình, Dịch vụ đưa...","Quảng trường, Cafe/quán bar","Công viên, Vườn hoa, Di tích, Núi",Cao
3,3423,Le Soleil Nha Trang Hotel,"Máy lạnh, Nhà hàng, Lễ tân 24h, Chỗ đậu xe, Th...","Quảng trường, Biển, Tháp, Khu mua sắm","Công viên, Nhà thờ, Bảo tàng, Tháp, Viện Hải d...",Thấp
4,435,Gold Ocean Apartments Nha Trang,"Máy lạnh, Nhà hàng, Hồ bơi, Lễ tân 24h, Chỗ đậ...","Biển, Quảng trường, Tháp, Khu mua sắm, Công viên","Nhà ga, Nhà thờ, Bảo tàng, Phòng tranh, Chợ, C...",Trung bình
...,...,...,...,...,...,...
9271,6918,Lotus Village Resort Mui Ne,"Máy lạnh, Nhà hàng, Hồ bơi, Lễ tân 24h, Chỗ đậ...",##,"Di tích, Bãi đá, Biển, Sân golf, Công viên, Nú...",Cao
9272,7327,The Secret Garden,"WiFi miễn phí, Xe đưa đón sân bay, Phòng g...","Cầu, Bảo tàng, Cafe/quán bar, Sông","Cầu, Cánh đồng, Khu giải trí, Công viên, Núi, ...",Thấp
9273,4135,Raon Dalat - STAY 24H,"WiFi miễn phí, Chỗ đỗ xe miễn phí, Phòn...",Cafe/quán bar,"Vườn hoa, Quảng trường, Bảo tàng, Công Viên, D...",Thấp
9274,10364,TH Quy Nhơn Hotel,"Xe đưa đón sân bay, Phòng không hút thuốc,...","Cafe/quán bar, Biển","Khu du lịch, Biển",Thấp


In [28]:
# Xử lí các giá trị trùng lặp khi tạo dataFrame
df_merge[['Facilities','Around', 'Vicinity', 'Price']] = df_merge[['Facilities','Around', 'Vicinity', 'Price']].apply(lambda x: x.apply(lambda y: ', '.join(set(y.split(', ')))))

# Tiền xử lí và trích xuất vecto đặc trưng của Khách sạn và người dùng

In [30]:
# Lấy bộ từ để đưa vào mô hình Word2vec
hotel_fac_corpus = hotel_data['Facilities'].tolist()
hotel_ar_corpus = hotel_data['Around'].tolist()
hotel_vici_corpus = hotel_data['Vicinity'].tolist()
hotel_pri_corpus = hotel_data['Price_Category'].tolist()
user_fac_corpus = df_merge['Facilities'].tolist()
user_ar_corpus = df_merge['Around'].tolist()
user_vici_corpus = df_merge['Vicinity'].tolist()
user_pri_corpus = df_merge['Price'].tolist()

In [29]:
# Khởi tạo 1 Lemmatizer
lemmatizer = WordNetLemmatizer()

In [31]:
def preprocess_text(text):
    #Loại bỏ dấu phẩy
    text = text.replace(',', '')

    # Tokennize các từ với pyvi
    tokens = ViTokenizer.tokenize(text)

    # Chuyển về viết thường
    tokens = tokens.lower().split()

    # Chuẩn hóa từ
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

In [42]:
def extract_document_vectors(corpus):
    # Xử lí dữ liệu đầu vào
    preprocessed_corpus = [preprocess_text(text) for text in corpus]

    # Huấn luyện mô hình với bộ từ được đưa vào
    model = Word2Vec(sentences=preprocessed_corpus, vector_size=300, window=10, min_count=1)

    # Trích xuất vecto đặc trưng
    document_vectors = []
    for text in preprocessed_corpus:
        doc_vector = []
        for word in text:
            if word in model.wv:
                doc_vector.append(model.wv[word])
        if doc_vector:
            document_vector = sum(doc_vector) / len(doc_vector)
            document_vectors.append(document_vector)

    return document_vectors

In [43]:
# Trích xuất vecto đặc trưng
hotel_facilities_vectors = np.array(extract_document_vectors(hotel_fac_corpus))
hotel_around_vectors = np.array(extract_document_vectors(hotel_ar_corpus))
hotel_vicinity_vectors = np.array(extract_document_vectors(hotel_vici_corpus))
hotel_price_vectors = np.array(extract_document_vectors(hotel_pri_corpus))
user_facilities_vectors = np.array(extract_document_vectors(user_fac_corpus))
user_around_vectors = np.array(extract_document_vectors(user_ar_corpus))
user_vicinity_vectors = np.array(extract_document_vectors(user_vici_corpus))
user_price_vectors = np.array(extract_document_vectors(user_pri_corpus))

# Tính độ tương đồng Cosine_similarity

In [44]:
# Tạo vecto đặc trưng của user và khách sạn bằng cách lấy trung bình các vecto thành phần
hotel_vec = 0.25*hotel_facilities_vectors + 0.25*hotel_around_vectors + 0.25*hotel_vicinity_vectors + 0.25*hotel_price_vectors
user_vec = 0.25*user_facilities_vectors + 0.25*user_around_vectors + 0.25*user_vicinity_vectors + 0.25*user_price_vectors

In [45]:
# Tính độ tương đồng cosine
sim_synthesis = cosine_similarity(user_vec,hotel_vec)

In [46]:
sim_synthesis

array([[0.71381307, 0.71366423, 0.71306944, ..., 0.7253521 , 0.71949774,
        0.7186388 ],
       [0.721679  , 0.7214062 , 0.72076035, ..., 0.7323262 , 0.72727126,
        0.72764707],
       [0.7759683 , 0.77582264, 0.77528113, ..., 0.78582126, 0.78089106,
        0.7809717 ],
       ...,
       [0.72092026, 0.72064674, 0.71997124, ..., 0.7311853 , 0.7264107 ,
        0.72672707],
       [0.6752041 , 0.67521334, 0.6745671 , ..., 0.68618155, 0.68064547,
        0.6798993 ],
       [0.73581755, 0.73579973, 0.73505956, ..., 0.7482066 , 0.74204683,
        0.741162  ]], dtype=float32)

# Đề xuất khách sạn

In [47]:
# Hàm đề xuất khách sạn cho người dùng
def get_recommendations_content_based(userid, num_hotels):
    # Lấy độ tương đồng với khách sạn của user
    user_index = df_merge[df_merge['IDuser'] == userid].index
    if not user_index.empty:
        sim_scores = list(enumerate(sim_synthesis[user_index[0]]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[:200]
        hotel_indices = [i[0] for i in sim_scores]
        similar_hotels = hotel_data.loc[hotel_indices, 'NameHotel'].tolist()

        # Lọc và lấy ra danh sách khách sạn đề đề xuất
        list_recommendation_location = []
        for hotel_name in similar_hotels:
            if hotel_name in hotel_data['NameHotel'].values:
                list_recommendation_location.append(hotel_name)
                if len(list_recommendation_location) == num_hotels:
                    break

        return list_recommendation_location
    else:
        return []

In [48]:
# Danh sách khách sạn đề xuất cho user có ID = 3857
%%time
ls = get_recommendations_content_based(3857, 20)
ls

CPU times: user 3.49 ms, sys: 0 ns, total: 3.49 ms
Wall time: 3.45 ms


['Melia Hồ Tràm Beach Resort',
 'Khu nghỉ dưỡng Risemount Premier Đà Nẵng',
 'The Palmy Phú Quốc Resort & Spa',
 'Seashells Phú Quốc Hotel & Spa',
 'Hội An Việt House Homestay',
 'Famiana Resort & Spa Phú Quốc',
 'Khách sạn Hải Âu Quy Nhơn',
 'Khu nghỉ dưỡng và Spa Mikazuki Nhật Bản',
 'Khu nghỉ dưỡng The Sailing Bay Beach Phan Thiết',
 'Khu nghỉ dưỡng Pilgrimage Village Boutique',
 'Khách sạn Fusion Suites Vũng Tàu',
 'Ly Ky Hotel',
 'Khách sạn Lasenta Boutique Hội An',
 'Swiss-Belresort Tuyền Lâm - Đà Lạt',
 'Khách sạn Zalo Sea',
 'Thanh Binh Central Hotel',
 'Cereja Hotel & Resort Đà Lạt',
 'Phan Thiet Ocean Dunes Resort',
 'Khách sạn Mường Thanh Luxury Phú Quốc',
 'Sandunes Beach Resort and Spa']

In [49]:
# Đề xuất cho toàn bộ các user
%%time
list_content_based = []
num_hotels_cb = 100

for i in tqdm(range(df_merge.shape[0])):
  list_content_based.append(get_recommendations_content_based(df_merge.iloc[i,0], num_hotels_cb))

100%|██████████| 9276/9276 [00:28<00:00, 327.45it/s]

CPU times: user 26.9 s, sys: 431 ms, total: 27.3 s
Wall time: 28.3 s


# Lưu kết quả vào để đánh giá mô hình

In [50]:
# Lưu kết quả khuyến nghị vào dataFrame
list_user = df_merge['IDuser']
df_pred = pd.DataFrame(list_user,columns=['IDuser'])
df_pred['List Hotel Recommended'] = list_content_based

In [51]:
# Xuất dataFrame ra file .csv
df_pred.to_csv('/content/drive/MyDrive/DS300_DoAn/data/List_Hotel_Recommend.csv')